In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [71]:
df = pd.read_csv('../data/clean/abnb_clean.gz', low_memory=False)

In [72]:
df = df.drop(columns = 'Unnamed: 0')
df['year'] = pd.to_datetime(df['last_scraped']).dt.to_period('Y')
df['availability_pct'] = df['availability_365']/365 *100
df["borough"] = df["borough"].astype("str")
df["borough"] = df["borough"].str.upper()

In [77]:
df

,id,last_scraped,host_id,host_name,neighborhood,latitude,longitude,room_type,bedrooms,availability_365,calculated_host_listings_count,geom,zipcode,borough,year,availability_pct
0,2539,2022-09-07,2787,John,Kensington,40.64529,-73.97238,Private room,1.0,356,9,"40.64529,-73.97238",11218,BROOKLYN,2022,97.534247
1,5121,2022-09-07,7356,Garon,Bedford-Stuyvesant,40.68535,-73.95512,Private room,1.0,335,2,"40.68535,-73.95512",11238,BROOKLYN,2022,91.780822
2,45910,2022-09-07,204539,Mark,Ridgewood,40.70309,-73.89963,Entire home/apt,5.0,365,6,"40.70309,-73.89963",11385,QUEENS,2022,100.000000
3,5136,2022-09-07,7378,Rebecca,Sunset Park,40.66265,-73.99454,Entire home/apt,2.0,179,1,"40.66265,-73.99454",11215,BROOKLYN,2022,49.041096
4,45935,2022-09-07,204586,L,Mott Haven,40.80635,-73.92201,Private room,1.0,83,1,"40.80635,-73.92201",10454,BRONX,2022,22.739726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335498,46937371,2020-01-01,27181542,Jt,Kips Bay,40.73961,-73.97971,Entire home/apt,NaN,180,15,"40.73961,-73.97971",11217,BROOKLYN,2020,49.315068
335499,46937954,2020-01-01,287807769,Allen,Upper West Side,40.78061,-73.98514,Entire home/apt,NaN,83,1,"40.78061,-73.98514",11226,BROOKLYN,2020,22.739726
335500,46938514,2020-01-01,303319010,Naiomy,East Harlem,40.80054,-73.93340,Shared room,NaN,176,1,"40.80054,-73.9334",11226,BROOKLYN,2020,48.219178
335501,46938537,2020-01-01,3351317,Clara,Kips Bay,40.74033,-73.98242,Entire home/apt,NaN,83,3,"40.74033,-73.98242",10115,MANHATTAN,2020,22.739726


In [95]:
df.to_csv("../data/clean/airbnb_clean.csv")